In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from matplotlib import pyplot as plt

pd.set_option('display.max_rows', None)

In [ ]:
#Open csv file.

data = pd.read_csv("/content/drive/MyDrive/NSQIP-ALIF/data_combined.csv", index_col=0, na_values = -99)

In [ ]:
#See all columns.

print(list(data.columns))

In [ ]:
#Check data shape.

data.shape

In [ ]:
#Define variables of interest (predictor variables, inclusion/exclusion criteria, outcomes of interest).

variables = ['SEX', 'RACE_NEW', 'ETHNICITY_HISPANIC', 'CPT', 'STILLINHOSP', 'TRANST', 'AGE', 'ANESTHES', 'SURGSPEC', 'ELECTSURG', 'HEIGHT', 'WEIGHT', 'DIABETES', 'SMOKE', 'DYSPNEA', 'FNSTATUS2', 'VENTILAT', 'HXCOPD', 'ASCITES', 'HXCHF', 'HYPERMED', 'RENAFAIL', 'DIALYSIS', 'DISCANCR', 'WNDINF', 'STEROID', 'WTLOSS', 'BLEEDDIS', 'TRANSFUS', 'PRSEPIS', 'PRSODM', 'PRBUN', 'PRCREAT', 'PRALBUM', 'PRBILI', 'PRSGOT', 'PRALKPH', 'PRWBC', 'PRHCT', 'PRPLATE', 'PRPTT', 'PRINR', 'PRPT', 'EMERGNCY', 'WNDCLAS', 'ASACLAS', 'PODIAG', 'PODIAG10', 'READMISSION1', 'REOPERATION1', 'DISCHDEST', 'TOTHLOS', 'NWNDINFD', 'NORGSPCSSI', 'NDEHIS', 'NREINTUB', 'NPULEMBOL', 'NFAILWEAN', 'NRENAINSF', 'NOPRENAFL', 'NCNSCVA', 'NCDARREST', 'NCDMI', 'NOTHBLEED', 'NOTHDVT', 'NOTHSYSEP', 'NOTHSESHOCK',  'OTHERCPT1', 'OTHERCPT2', 'OTHERCPT3', 'OTHERCPT4', 'OTHERCPT5', 'OTHERCPT6', 'OTHERCPT7', 'OTHERCPT8', 'OTHERCPT9', 'OTHERCPT10', 'CONCPT1', 'CONCPT2', 'CONCPT3', 'CONCPT4', 'CONCPT5', 'CONCPT6', 'CONCPT7', 'CONCPT8', 'CONCPT9', 'CONCPT10']

In [ ]:
#Remove unwanted columns and check data shape.

data = data[variables]

data.shape

#Inclusion Criteria

In [ ]:
#Apply inclusion criteria for elective surgeries.

before = data.shape[0]
data = data[(data['ELECTSURG'] == 'Yes')]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Apply inclusion criteria for general anesthesia.

before = data.shape[0]
data = data[(data['ANESTHES'] == 'General')]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Apply inclusion criteria for surgical specialties.

before = data.shape[0]
data = data[(data['SURGSPEC'] == 'Neurosurgery') | (data['SURGSPEC'] == 'Orthopedics')]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

#Exclusion Criteria

##Other Exclusion Criteria

In [ ]:
#Apply exclusion criteria for emergency surgery.

before = data.shape[0]
data = data[(data['EMERGNCY'] == 'No')]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Apply exclusion criteria for wound class.

before = data.shape[0]
data = data[(data['WNDCLAS'] == '1-Clean')]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Apply exclusion criteria for preoperative sepsis.

before = data.shape[0]
data = data[(data['PRSEPIS'] == 'None')]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Apply exclusion criteria for ASA class.

before = data.shape[0]
data = data[(data['ASACLAS'] != '4-Life Threat') & (data['ASACLAS'] != '5-Moribund') & (data['ASACLAS'] != 'None assigned')]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Apply exclusion criteria for patients still in hospital after 30 days.

before = data.shape[0]
data = data[(data['STILLINHOSP'] == 'No')]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Apply exclusion criteria for patients with missing LOS and discharge destinations.

before = data.shape[0]
data = data[data['TOTHLOS'].notna()]
data = data[data['DISCHDEST'].notna()]
data = data[(data['DISCHDEST'] != 'Unknown')]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Apply exclusion criteria for discharge destinations.

before = data.shape[0]
data = data[(data['DISCHDEST'] != 'Expired')]
data = data[(data['DISCHDEST'] != 'Hospice')]
data = data[(data['DISCHDEST'] != 'Against Medical Advice (AMA)')]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

##Based on CPT Codes

In [ ]:
#Exclude patients with CPT codes that were assigned to be excluding.

columns = ['CPT', 'OTHERCPT1', 'OTHERCPT2', 'OTHERCPT3', 'OTHERCPT4', 'OTHERCPT5', 'OTHERCPT6',
           'OTHERCPT7', 'OTHERCPT8', 'OTHERCPT9', 'OTHERCPT10', 'CONCPT1', 'CONCPT2', 'CONCPT3',
           'CONCPT4', 'CONCPT5', 'CONCPT6', 'CONCPT7', 'CONCPT8', 'CONCPT9', 'CONCPT10']

codes_to_exclude = [22612, 22800, 22802, 22804, 22842, 22843, 22844, 63030, 63047, 63042]

for column in columns:
    for code in codes_to_exclude:
        data.loc[data[column] == code, 'CPT_EX'] = 'Yes'

data.loc[data['CPT_EX'] != 'Yes', 'CPT_EX'] = 'No'

before = data.shape[0]
data = data[(data['CPT_EX'] == 'No')]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

##Based on ICD Codes

In [ ]:
#See the ICD codes' unique value counts for the patient cohort.

icd_codes_df = data['PODIAG10'].value_counts(normalize=False, dropna=False).to_frame()
icd_codes_index = icd_codes_df.index.tolist()

In [ ]:
#Exclude patients with ICD codes that were used less than 10 in the patient population.

icd_codes_df = data['PODIAG10'].value_counts(normalize=False, dropna=False).to_frame()

icd_codes_df.columns =['Value']

icd_codes_df = icd_codes_df[icd_codes_df['Value'] >= 50]

icd_to_include = icd_codes_df.index.tolist()

before = data.shape[0]
data = data[data.PODIAG10.isin(icd_to_include)]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Get the descriptions for ICD codes.

icd10 = pd.read_csv("/content/drive/MyDrive/NSQIP-PUFs/ICD10_Descriptions.csv", index_col = 'PODIAG10', encoding = 'latin1', low_memory = False)
icd10 = icd10.filter(items = icd_to_include, axis=0)

In [ ]:
#Save the ICD codes with descriptions and value counts.

icd = pd.concat([icd10, icd_codes_df], axis=1)
icd.to_csv('/content/drive/MyDrive/NSQIP-ALIF/icd_value_counts.csv')

Stop and review the icd_value_counts.csv file.

In [ ]:
#Review the ICD table for excluding patients with exclude patients diagnosed with a fracture, neoplasm, infection, instrumentation related complications or lumbar/thoracic/sacral site diagnoses.

icd_to_exclude = ['M96.0', 'M96.1']

data = data[~data.PODIAG10.isin(icd_to_exclude)]

In [ ]:
#Drop patients with missing ICD codes.

before = data.shape[0]
data = data[data['PODIAG10'].notna()]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

#Renaming and Merging Response Values

In [ ]:
#Capitalize response values for 'SEX'.

data.loc[data['SEX'] == 'male', 'SEX'] = 'Male'
data.loc[data['SEX'] == 'female', 'SEX'] = 'Female'
data.loc[data['SEX'] == 'non-binary', 'SEX'] = 'Non-binary'

In [ ]:
#Check data for race.

data['RACE_NEW'].value_counts(normalize=False, dropna=False)

In [ ]:
#Simplify 'RACE_NEW' column.

data.loc[data['RACE_NEW'] == 'Unknown/Not Reported', 'RACE_NEW'] = 'Other/Unknown'
data.loc[data['RACE_NEW'] == 'Some Other Race', 'RACE_NEW'] = 'Other/Unknown'
data.loc[data['RACE_NEW'] == 'Native Hawaiian or Other Pacific Islander', 'RACE_NEW'] = 'Other/Unknown'
data.loc[data['RACE_NEW'] == 'American Indian or Alaska Native', 'RACE_NEW'] = 'Other/Unknown'
data.loc[data['RACE_NEW'] == 'Native Hawaiian or Pacific Islander', 'RACE_NEW'] = 'Other/Unknown'
data.loc[data['RACE_NEW'] == 'Native Hawaiian or Other Pacific Islander', 'RACE_NEW'] = 'Other/Unknown'
data.loc[data['RACE_NEW'] == 'Race combinations with low frequency', 'RACE_NEW'] = 'Other/Unknown'

data['RACE_NEW'].value_counts(normalize=False, dropna=False)

In [ ]:
#Convert inch to meter and lbs to kg.

lbs_to_kg_ratio = 0.453592
inch_to_meter_ratio = 0.0254

data['HEIGHT'] *= inch_to_meter_ratio
data['WEIGHT'] *= lbs_to_kg_ratio

In [ ]:
#Check data for transfer status.

data['TRANST'].value_counts(normalize=False, dropna=False)

In [ ]:
#Simplify 'TRANST' column.

data.loc[data['TRANST'] == 'Not transferred (admitted from home)', 'TRANST'] = 'Not transferred'
data.loc[data['TRANST'] == 'From acute care hospital inpatient', 'TRANST'] = 'Transferred'
data.loc[data['TRANST'] == 'Outside emergency department', 'TRANST'] = 'Transferred'
data.loc[data['TRANST'] == 'Nursing home - Chronic care - Intermediate care', 'TRANST'] = 'Transferred'
data.loc[data['TRANST'] == 'Transfer from other', 'TRANST'] = 'Transferred'

data['TRANST'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for dyspnea.

data['DYSPNEA'].value_counts(normalize=False, dropna=False)

In [ ]:
#Simplify 'DYSPNEA' column.

data.loc[data['DYSPNEA'] == 'No', 'DYSPNEA'] = 'No'
data.loc[data['DYSPNEA'] == 'MODERATE EXERTION', 'DYSPNEA'] = 'Yes'
data.loc[data['DYSPNEA'] == 'AT REST', 'DYSPNEA'] = 'Yes'

data['DYSPNEA'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for diabetes status.

data['DIABETES'].value_counts(normalize=False, dropna=False)

In [ ]:
#Simplify 'DIABETES' column.

data.loc[data['DIABETES'] == 'NO', 'DIABETES'] = 'No'
data.loc[data['DIABETES'] == 'NON-INSULIN', 'DIABETES'] = 'Yes'
data.loc[data['DIABETES'] == 'INSULIN', 'DIABETES'] = 'Yes'

data['DIABETES'].value_counts(normalize=False, dropna=False)

In [ ]:
#Convert 90+ to 91 and AGE column to integer.

data.loc[data['AGE'] == '90+', 'AGE'] = 91
data['AGE'] = pd.to_numeric(data['AGE'], downcast='integer')

In [ ]:
#Identify multiple level surgeries with the new column 'Single or Multiple Level Surgery'.

columns = ['CPT', 'CONCPT1', 'CONCPT2', 'CONCPT3', 'CONCPT4', 'CONCPT5', 'CONCPT6',
           'CONCPT7', 'CONCPT9', 'CONCPT10', 'OTHERCPT1', 'OTHERCPT2', 'OTHERCPT3',
           'OTHERCPT4', 'OTHERCPT5', 'OTHERCPT6', 'OTHERCPT7', 'OTHERCPT9', 'OTHERCPT10']

codes = [22585]

for col in columns:
    for code in codes:
        data.loc[data[col] == code, 'Single or Multiple Level Surgery'] = 'Multiple'

data.loc[data['Single or Multiple Level Surgery'] != 'Multiple', 'Single or Multiple Level Surgery'] = 'Single'

data['Single or Multiple Level Surgery'].value_counts(dropna=False)

In [ ]:
#Define major complications.

complications = ['NWNDINFD', 'NORGSPCSSI', 'NDEHIS', 'NREINTUB', 'NPULEMBOL',
                 'NFAILWEAN', 'NRENAINSF', 'NOPRENAFL', 'NCNSCVA', 'NCDARREST',
                 'NCDMI', 'NOTHBLEED', 'NOTHDVT', 'NOTHSYSEP', 'NOTHSESHOCK']

#Define a new column 'MAJRCOMP' to hold the sum of all major complications.

data['MAJRCOMP'] = data[complications].sum(axis=1)
data['MAJRCOMP'].fillna(0, inplace=True)

#Convert 'MAJRCOMP' into a categorical variable.

data.loc[data['MAJRCOMP'] == 0, 'COMPLICATIONS'] = 'No'
data.loc[data['MAJRCOMP'] >= 1, 'COMPLICATIONS'] = 'Yes'

data['COMPLICATIONS'].value_counts(normalize=False, dropna=False)

In [ ]:
#See all columns.

print(list(data.columns))

In [ ]:
#Drop unwanted columns.

drop = ['STILLINHOSP', 'ANESTHES', 'EMERGNCY', 'WNDCLAS', 'ELECTSURG', 'PRSEPIS', 'MAJRCOMP', 'NWNDINFD', 'NORGSPCSSI', 'NDEHIS', 'NREINTUB', 'NPULEMBOL', 'NFAILWEAN', 'NRENAINSF', 'NOPRENAFL', 'NCNSCVA', 'NCDARREST', 'NCDMI', 'NOTHBLEED', 'NOTHDVT', 'NOTHSYSEP', 'NOTHSESHOCK', 'PODIAG', 'PODIAG10', 'CPT_EX', 'CPT', 'OTHERCPT1', 'OTHERCPT2', 'OTHERCPT3', 'OTHERCPT4', 'OTHERCPT5', 'OTHERCPT6', 'OTHERCPT7', 'OTHERCPT8', 'OTHERCPT9', 'OTHERCPT10', 'CONCPT1', 'CONCPT2', 'CONCPT3', 'CONCPT4', 'CONCPT5', 'CONCPT6', 'CONCPT7', 'CONCPT8', 'CONCPT9', 'CONCPT10']

data.drop(drop, axis=1, inplace=True)

#Imputation

In [ ]:
#See all columns.

print(list(data.columns))

In [ ]:
#Define numerical and categorical columns.

num_cols = list(data.select_dtypes('number').columns)
print('Numerical columns: {}'.format(num_cols), '\n')

cat_cols = list(data.select_dtypes('object').columns)
print('Categorical columns: {}'.format(cat_cols))

In [ ]:
#Remove outcomes.

num_cols_remove = ['TOTHLOS']
cat_cols_remove = ['READMISSION1', 'DISCHDEST', 'COMPLICATIONS', 'REOPERATION1']

num_cols = [i for i in num_cols if i not in num_cols_remove]
cat_cols = [i for i in cat_cols if i not in cat_cols_remove]

In [ ]:
#Check missing values for numerical columns.

missing_num = data[num_cols].isnull().mean().round(4).mul(100).sort_values(ascending=False)

print(missing_num, '\n')

missing_num = pd.DataFrame(missing_num)

missing_num.columns = ['Value']

missing_num = missing_num[missing_num['Value'] > 0]

print('Numerical variables with missing values: ', list(missing_num.index), '\n')

print('Number of numerical variables with missing values: ', len(list(missing_num.index)), '\n')

missing_num = missing_num[missing_num['Value'] > 25]

missing_num = list(missing_num.index)

print('Excluded numerical variables: ', missing_num)

In [ ]:
#Drop numerical columns with missing values over 25%.

data.drop(missing_num, axis=1, inplace=True)

In [ ]:
#Define new numerical columns.

num_cols = [x for x in num_cols if x not in missing_num]

In [ ]:
#Impute missing numerical values.

num_imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
data[num_cols] = num_imputer.fit_transform(data[num_cols])

In [ ]:
#Check missing values for categorical columns.

missing_cat = data[cat_cols].isnull().mean().round(4).mul(100).sort_values(ascending=False)

print(missing_cat, '\n')

missing_cat = pd.DataFrame(missing_cat)

missing_cat.columns = ['Value']

missing_cat = missing_cat[missing_cat['Value'] > 0]

print('Categorical variables with missing values: ', list(missing_cat.index), '\n')

print('Number of categorical variables with missing values: ', len(list(missing_cat.index)), '\n')

missing_cat = missing_cat[missing_cat['Value'] > 25]

missing_cat = list(missing_cat.index)

print('Excluded categorical variables: ', missing_cat)

In [ ]:
#Drop categorical columns with missing values over 25%.

data.drop(missing_cat, axis=1, inplace=True)

In [ ]:
#Define new categorical columns.

cat_cols = [x for x in cat_cols if x not in missing_cat]

In [ ]:
#Replace missing categorical values with 'Unknown'.

for col in cat_cols:
    data[col].fillna(value='Unknown', inplace=True)

#Final Touches

In [ ]:
#Change variable names to field names.

data_dictionary = pd.read_csv("/content/drive/MyDrive/NSQIP-PUFs/Data_Dictionary.csv", encoding = 'latin1', index_col = None, low_memory = False)
FieldNames = dict(zip(data_dictionary['Variable'], data_dictionary['Field Name']))
data.columns = data.columns.map(lambda x: FieldNames.get(x, x))

In [ ]:
#Save imputed data.

data.to_csv('/content/drive/MyDrive/NSQIP-ALIF/imputed_data.csv')

In [ ]:
#Manual label encoding.

data.loc[data['Sex'] == 'Male', 'Sex'] = 0
data.loc[data['Sex'] == 'Female', 'Sex'] = 1
data.loc[data['Sex'] == 'Non-binary', 'Sex'] = 2
data.loc[data['Race'] == 'White', 'Race'] = 0
data.loc[data['Race'] == 'Black or African American', 'Race'] = 1
data.loc[data['Race'] == 'Asian', 'Race'] = 2
data.loc[data['Race'] == 'American Indian or Alaska Native', 'Race'] = 3
data.loc[data['Race'] == 'Native Hawaiian or Pacific Islander', 'Race'] = 4
data.loc[data['Race'] == 'Other/Unknown', 'Race'] = 4
data.loc[data['Hispanic Ethnicity'] == 'No', 'Hispanic Ethnicity'] = 0
data.loc[data['Hispanic Ethnicity'] == 'Yes', 'Hispanic Ethnicity'] = 1
data.loc[data['Hispanic Ethnicity'] == 'Unknown', 'Hispanic Ethnicity'] = 2
data.loc[data['Transfer Status'] == 'Not transferred', 'Transfer Status'] = 0
data.loc[data['Transfer Status'] == 'Transferred', 'Transfer Status'] = 1
data.loc[data['Transfer Status'] == 'Unknown', 'Transfer Status'] = 2
data.loc[data['Surgical Specialty'] == 'Neurosurgery', 'Surgical Specialty'] = 0
data.loc[data['Surgical Specialty'] == 'Orthopedics', 'Surgical Specialty'] = 1
data.loc[data['Diabetes Mellitus Requiring Therapy'] == 'No', 'Diabetes Mellitus Requiring Therapy'] = 0
data.loc[data['Diabetes Mellitus Requiring Therapy'] == 'Yes', 'Diabetes Mellitus Requiring Therapy'] = 1
data.loc[data['Current Smoker Status'] == 'No', 'Current Smoker Status'] = 0
data.loc[data['Current Smoker Status'] == 'Yes', 'Current Smoker Status'] = 1
data.loc[data['Current Smoker Status'] == 'Unknown', 'Current Smoker Status'] = 2
data.loc[data['Dyspnea'] == 'No', 'Dyspnea'] = 0
data.loc[data['Dyspnea'] == 'Yes', 'Dyspnea'] = 1
data.loc[data['Ventilator Dependency'] == 'No', 'Ventilator Dependency'] = 0
data.loc[data['Ventilator Dependency'] == 'Yes', 'Ventilator Dependency'] = 1
data.loc[data['Functional Status'] == 'Independent', 'Functional Status'] = 0
data.loc[data['Functional Status'] == 'Partially Dependent', 'Functional Status'] = 1
data.loc[data['Functional Status'] == 'Totally Dependent', 'Functional Status'] = 2
data.loc[data['Functional Status'] == 'Unknown', 'Functional Status'] = 3
data.loc[data['History of Severe COPD'] == 'No', 'History of Severe COPD'] = 0
data.loc[data['History of Severe COPD'] == 'Yes', 'History of Severe COPD'] = 1
data.loc[data['Ascites within 30 Days Prior to Surgery'] == 'No', 'Ascites within 30 Days Prior to Surgery'] = 0
data.loc[data['Ascites within 30 Days Prior to Surgery'] == 'Yes', 'Ascites within 30 Days Prior to Surgery'] = 1
data.loc[data['CHF within 30 Days Prior to Surgery'] == 'No', 'CHF within 30 Days Prior to Surgery'] = 0
data.loc[data['CHF within 30 Days Prior to Surgery'] == 'Yes', 'CHF within 30 Days Prior to Surgery'] = 1
data.loc[data['Hypertension Requiring Medication'] == 'No', 'Hypertension Requiring Medication'] = 0
data.loc[data['Hypertension Requiring Medication'] == 'Yes', 'Hypertension Requiring Medication'] = 1
data.loc[data['Acute Renal Failure'] == 'No', 'Acute Renal Failure'] = 0
data.loc[data['Acute Renal Failure'] == 'Yes', 'Acute Renal Failure'] = 1
data.loc[data['Currently Requiring or on Dialysis'] == 'No', 'Currently Requiring or on Dialysis'] = 0
data.loc[data['Currently Requiring or on Dialysis'] == 'Yes', 'Currently Requiring or on Dialysis'] = 1
data.loc[data['Disseminated Cancer'] == 'No', 'Disseminated Cancer'] = 0
data.loc[data['Disseminated Cancer'] == 'Yes', 'Disseminated Cancer'] = 1
data.loc[data['Open Wound'] == 'No', 'Open Wound'] = 0
data.loc[data['Open Wound'] == 'Yes', 'Open Wound'] = 1
data.loc[data['Steroid or Immunosuppressant for a Chronic Condition'] == 'No', 'Steroid or Immunosuppressant for a Chronic Condition'] = 0
data.loc[data['Steroid or Immunosuppressant for a Chronic Condition'] == 'Yes', 'Steroid or Immunosuppressant for a Chronic Condition'] = 1
data.loc[data['Malnourishment'] == 'No', 'Malnourishment'] = 0
data.loc[data['Malnourishment'] == 'Yes', 'Malnourishment'] = 1
data.loc[data['Bleeding Disorder'] == 'No', 'Bleeding Disorder'] = 0
data.loc[data['Bleeding Disorder'] == 'Yes', 'Bleeding Disorder'] = 1
data.loc[data['RBC Transfusion within 72 Hours Prior to Surgery'] == 'No', 'RBC Transfusion within 72 Hours Prior to Surgery'] = 0
data.loc[data['RBC Transfusion within 72 Hours Prior to Surgery'] == 'Yes', 'RBC Transfusion within 72 Hours Prior to Surgery'] = 1
data.loc[data['ASA Classification'] == '1-No Disturb', 'ASA Classification'] = 0
data.loc[data['ASA Classification'] == '2-Mild Disturb', 'ASA Classification'] = 1
data.loc[data['ASA Classification'] == '3-Severe Disturb', 'ASA Classification'] = 2

data.loc[data['Single or Multiple Level Surgery'] == 'Single', 'Single or Multiple Level Surgery'] = 0
data.loc[data['Single or Multiple Level Surgery'] == 'Multiple', 'Single or Multiple Level Surgery'] = 1

columns = ['Sex', 'Race', 'Hispanic Ethnicity', 'Transfer Status', 'Surgical Specialty',
           'Diabetes Mellitus Requiring Therapy', 'Current Smoker Status', 'Dyspnea',
           'Ventilator Dependency', 'Functional Status', 'History of Severe COPD',
           'Ascites within 30 Days Prior to Surgery', 'CHF within 30 Days Prior to Surgery',
           'Hypertension Requiring Medication', 'Acute Renal Failure',
           'Currently Requiring or on Dialysis', 'Disseminated Cancer', 'Open Wound',
           'Steroid or Immunosuppressant for a Chronic Condition', 'Malnourishment',
           'Bleeding Disorder', 'RBC Transfusion within 72 Hours Prior to Surgery',
           'ASA Classification', 'Single or Multiple Level Surgery']

for column in columns:
    data[column] = data[column].astype(int)

In [ ]:
#Save final data.

data.to_csv('/content/drive/MyDrive/NSQIP-ALIF/final_data.csv')

In [ ]:
#Save data for Gradio.

gradio = data.copy()

gradio.columns = gradio.columns.str.replace(' ', '_', regex=True)
gradio.columns = gradio.columns.str.replace('-', '', regex=True)

gradio.to_csv('/content/drive/MyDrive/NSQIP-ALIF/gradio_data.csv')